In [2]:
!pip install langchain
!pip install langchain-openai


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
from dotenv import load_dotenv

load_dotenv() 
openai_api_key = os.getenv("OPENAI_API_KEY")


In [8]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate,  HumanMessagePromptTemplate,  PipelinePromptTemplate, FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser

# 기본 LLM



*   ChatOpenAI : 모델을 불러오는 클래스
*   ChatPromptTemplate : prompt 템플릿을 제공해주는 클래스
*   ChatPromptTemplate.from_template() : 문자열 형태의 템플릿을 인자로 받아, 해당 형식에 맞는 프롬프트 객체를 생성
*   StrOutputParaser : 모델을 출력을 문자열 형태로 파싱하여 최종 결과를 반환
*   invoke : chain을 실행하는 메서드



In [9]:
# 가장 간단한 LLM 실행
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

llm.invoke("지구의 자전 주기는?")
# llm.invoke("지구의 자전 주기는?").content # 출력 내용만 가져옴.

AIMessage(content='지구의 자전 주기는 약 24시간입니다. 이것은 하루의 길이와 같은 시간을 의미합니다. 하루의 길이는 일출과 일몰이 발생하는 시간에 영향을 줍니다. 지구는 자전하는 동안 태양 주위를 공전하면서 자전하는 데 걸리는 시간은 약 24시간입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 16, 'total_tokens': 128, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbLYIHykXSFR5LTkf8Viuhg3nh8IO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--2bccd0db-17bf-46ea-a25c-ed1524494e54-0', usage_metadata={'input_tokens': 16, 'output_tokens': 112, 'total_tokens': 128, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [7]:
# prompt template 도입
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
prompt

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in astronomy. Answer the question. <Question>: {input}'), additional_kwargs={})])

In [8]:
# Chain으로 연결
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = prompt | llm

chain.invoke({"input": "지구의 자전 주기는?"})

AIMessage(content='지구의 자전 주기는 약 24시간, 즉 하루입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 30, 'total_tokens': 53, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BbHkdZFp5WmG8K3gBc6gXtCC4du0N', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--adae98db-ebb9-4264-9352-0638be3cbc40-0', usage_metadata={'input_tokens': 30, 'output_tokens': 23, 'total_tokens': 53, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [9]:
# STrOutputParser 도입.
prompt = ChatPromptTemplate.from_template("You are an expert in astronomy. Answer the question. <Question>: {input}")
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
output_parser = StrOutputParser()

chain = prompt | llm | output_parser

chain.invoke({"input": "지구의 자전 주기는?"})


'지구의 자전 주기는 약 24시간입니다. 이는 하루 동안 지구가 자전하는 시간을 나타내며, 우리가 낮과 밤을 경험하는 이유입니다.'

In [16]:
# 순차적인 체인 연결

prompt1 = ChatPromptTemplate.from_template("translates {korean_word} to English.")
prompt2 = ChatPromptTemplate.from_template(
    "explain {english_word} using oxford dictionary to me in Korean."
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    {"english_word": chain1}
    | prompt2
    | llm
    | StrOutputParser()
)

chain2.invoke({"korean_word":"미래"})

'According to the Oxford Dictionary, the word \'future\' is defined as "the time or a period of time following the moment of speaking or writing; time regarded as still to come." This definition aligns with the concept of \'미래\' in Korean, as it also refers to a time that is yet to come.'

# 프롬프트 작성 규칙

In [17]:
# 명확성과 구체성
question1 = "다음 주 주식 시장에 영향을 줄 수 있는 예정된 이벤트들은 무엇일까요?"
question2 = "주식 시장에 대해 알려주세요."

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

1. 기업 실적 발표: 다음 주에는 여러 기업들이 1분기 실적을 발표할 예정이며, 이로 인해 해당 기업의 주가에 영향을 줄 수 있습니다.

2. 미국 중간 선거: 미국의 중간 선거가 예정되어 있으며, 이는 미국 주식 시장에 영향을 줄 수 있습니다.

3. 경제 지표 발표: 다음 주에는 여러 나라에서 경제 지표가 발표될 예정이며, 이는 시장의 건강 상태에 대한 징후로 작용할 수 있습니다.

4. 세계 경제 상황: 전 세계의 경제 상황이 계속해서 변화하고 있으며, 다음 주에 예상치 못한 사건이 발생할 수 있습니다. 이는 글로벌 시장에 영향을 줄 수 있습니다.
------------------------------
주식 시장은 회사가 주식을 발행하여 자금을 조달하고, 이를 투자하는 투자자들이 거래하는 곳을 말합니다. 일반적으로는 증권사나 주식거래소를 통해 거래가 이루어지며, 투자자들은 회사의 경영상태나 성과에 기반해 주식을 매수 또는 매도하여 수익을 얻을 수 있습니다.

주식 시장은 주식시장의 종류에 따라 국내시장과 해외시장으로 나눌 수 있습니다. 국내시장에서는 한국거래소(KRX)를 중심으로 코스피와 코스닥이 대표적인 주식시장이며, 해외시장에서는 미국의 뉴욕증시나 나스닥, 일본의 도쿄증시 등이 대표적인 주식시장입니다.

주식 시장에서는 주식뿐만 아니라 채권, 파생상품 등 다양한 금융상품을 거래할 수 있으며, 주식 시장의 변동은 경제 상황이나 기업 실적, 정치적 요인 등 다양한 요인에 영향을 받습니다. 주식 시장은 투기와 투자의 장으로 평가되며, 적절한 정보 확보와 투자 전략을 세우는 것이 중요합니다.


In [ ]:
# 배경 정보를 포함
question1 = "2020년 미국 대선의 결과를 바탕으로 현재 정치 상황에 대한 분석을 해주세요."
question2 = "현재 정치 상황을 분석을 해주세요."

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

In [18]:
# 간결함.
question1 = "2021년에 발표된 삼성전자의 ESG 보고서를 요약해주세요."
question2 = "삼성전자의 다양한 보고서 중 특히 2021년에 발표된 ESG 보고서를 포함하여 전반적인 내용을 요약해주세요."

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

2021년 삼성전자의 ESG 보고서에는 기업의 환경, 사회 및 지배구조에 대한 다양한 정보와 노력들이 담겨 있습니다. 삼성전자는 지속가능한 경영을 위해 다양한 사업 분야에서 ESG 관련 활동을 펼치고 있습니다.

환경 측면에서는 기후변화 대응을 위해 온실가스 감축을 위한 목표를 설정하고 환경친화 제품 개발에 힘쓰고 있습니다. 또한 재활용을 촉진하고 환경 파괴를 최소화하기 위한 다양한 노력을 기울이고 있습니다.

사회 측면에서는 다양성과 평등을 증진하기 위해 다양한 채용 프로그램과 교육 지원을 실시하고 있습니다. 또한 지역사회에 기여하기 위해 다양한 사회공헌 활동을 전개하고 있습니다.

지배구조 측면에서는 투명하고 효과적인 기업 운영을 위해 총수회의체를 통해 의사결정을 내리고 이사회의 독립성을 강화하고 있습니다.

이처럼 삼성전자는 ESG 관심사에 대해 다양한 노력을 기울이고 지속가능한 경영을 위해 노력하고 있습니다.
------------------------------
삼성전자의 2021년 ESG 보고서는 환경, 사회, 지배구조에 대한 다양한 정보를 담고 있습니다. 환경 부문에서는 기후변화 대응을 위한 친환경 제품 및 기술 개발, 탄소 중립 전략, 재생에너지 사용 등의 노력을 소개하고 있습니다. 사회 부문에서는 다양성과 포용을 증진하기 위한 다양한 프로그램과 사회공헌 활동, 고객 개인정보 보호 및 사이버 보안에 대한 노력 등을 다루고 있습니다. 지배구조 부문에서는 투명하고 효과적인 기업 운영을 위한 다양한 정책과 시스템을 소개하고 있습니다. 총회사의 ESG 실천과 성과를 종합적으로 평가하고 향후 방향을 제시하는데 있어 중요한 자료로 활용될 것입니다.


In [19]:
# 열린 질문 사용
question1 = "신재생에너지에 대한 최신 연구 동향은 무엇인가요?."
question2 = "신재생에너지가 좋은가요?"

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

신재생에너지 분야에서의 최신 연구 동향은 다음과 같습니다:

1. 태양광 에너지: 태양전지의 효율을 높이기 위한 연구가 활발히 진행 중이며, 특히 페로브스카이트 태양전지와 페로브스카이트-신선형 태양전지 등의 기술이 주목받고 있습니다. 또한, 태양광 발전 소자의 경량화와 쉬운 설치를 위한 기술 개발이 진행 중에 있습니다.

2. 풍력 에너지: 풍력터빈의 효율을 높이고 운전 및 유지보수 비용을 줄이기 위한 연구가 진행 중에 있습니다. 또한, 해상 풍력 발전을 위한 새로운 기술과 설비가 개발되고 있습니다.

3. 지열 에너지: 친환경적이고 안정적인 지열 에너지를 활용하기 위한 지열 발전기술의 개선이 이루어지고 있습니다. 지열 열교환소 및 지열시축발전 시스템 등의 기술이 발전 중에 있습니다.

4. 바이오매스 에너지: 바이오매스를 이용한 전기 생산 및 열 생산 기술의 향상이 이뤄지고 있습니다. 고체연료전지와 생물다양성을 보존하면서 바이오매스를 이용한 에너지생산 기술이 주목받고 있습니다.

이러한 연구 결과를 토대로 신재생에너지 분야의 기술이 계속 발전하고 있으며, 미래에는 더욱 효율적이고 지속가능한 에너지 시스템이 구축될 것으로 기대됩니다.
------------------------------
신재생에너지는 지속가능하고 친환경적인 에너지원으로 인정받고 있습니다. 석탄, 석유와 같은 화석연료에 비해 대기오염, 기후변화 등에 미치는 영향이 적을 뿐만 아니라 고립된 지역에서도 발전이 가능하다는 장점이 있습니다. 또한, 재생에너지는 에너지 공급의 안정성을 높일 수 있으며, 지역 경제에 이바지할 수 있는 등 다양한 장점이 있습니다. 이러한 이유로 신재생에너지는 좋은 에너지원으로 인식되고 있습니다.


In [20]:
# 명확한 목표 설정
question1 = "AI 윤리에 대한 문제점과 해결 방안을 요약하여 설명해주세요."
question2 = "AI 윤리에 대해 이야기해주세요."

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

AI 윤리에 대한 문제점은 다음과 같습니다. 먼저, AI 시스템이 인간의 편견과 선입견을 반영할 수 있어서 편향된 결정을 내릴 수 있다는 점이 문제점입니다. 또한 AI 시스템이 개인정보 보호, 개인의 자유와 권리를 침해할 수도 있습니다. 또한 AI가 인간의 일자리를 대체하고, 사람들의 신뢰를 훼손시킬 수도 있습니다.

이러한 문제점을 해결하기 위한 방안으로는 먼저, 편향 데이터를 사용하지 않고 다양한 데이터를 활용하여 AI 시스템을 학습시키는 것이 중요합니다. 또한, AI 시스템을 설계할 때 윤리적인 가이드라인과 프레임워크를 고려하여 개발하는 것이 필요합니다. 또한, 인간의 관여와 감독이 중요하며, 투명성과 책임감 있는 운영이 필요합니다. 마지막으로, 개인정보 보호와 권리를 보호하기 위한 법률과 규정의 제고가 필요합니다. 이러한 방안을 통해 AI 윤리에 대한 문제를 해결할 수 있을 것으로 기대됩니다.
------------------------------
AI 윤리는 인공지능 기술이 사회적, 도덕적으로 적절하게 이용되어야 한다는 개념을 의미합니다. 

먼저, AI 윤리는 개인정보 보호와 권리를 존중해야 합니다. 개인정보를 수집, 저장, 처리할 때에는 사용자의 동의를 받아야 하고, 그 정보를 안전하게 보호해야 합니다. 

또한, AI 기술을 사용할 때에는 편향성과 차별을 방지해야 합니다. 데이터나 알고리즘 내에 내재된 편향을 인식하고 수정하여 공정하고 다양한 결정을 내릴 수 있도록 해야 합니다. 

또한, AI가 인간을 대신하여 결정을 내릴 때에는 책임과 투명성이 보장되어야 합니다. 결정이 내릴 때의 근거와 과정이 설명 가능해야 하고, 만일 문제가 발생했을 때 책임을 물을 수 있어야 합니다. 

AI 윤리는 인간 중심의 기술 개발을 지향합니다. 즉, 인공지능 기술이 인간을 위해 있는 것이라는 사고방식을 가지고, 사람들의 복지와 행복을 증진시키는 방향으로 사용되어야 한다는 점을 강조합니다. 

마지막으로, AI 윤리는 지속적인 논의와 업데이트가 필요합니다. 기술과

In [21]:
# 언어와 문체
question1 = "너는 미국 경제에 대해 분석하는 월스트리트 경제 전문가야. 미국 경제 전망에 대한 전문적인 분석을 부탁드립니다."
question2 = "미국 경제에 대해 말해봐"

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain1 = llm | StrOutputParser()
chain2 = llm | StrOutputParser()

print(chain1.invoke(question1))
print("-" * 30)
print(chain2.invoke(question2))

미국 경제에 대한 전망을 분석하기 전에 현재의 경제 상황을 살펴보겠습니다. 현재 미국은 코로나19 팬데믹으로 인해 경제적으로 어려움을 겪고 있습니다. 많은 기업들이 폐업하거나 일자리를 잃었고, 소비자 신뢰도가 낮아지는 등의 문제가 발생하고 있습니다.

그러나 최근 경제 회복 조짐도 보이고 있습니다. 백신 접종률이 상승하면서 경제 활동이 원활해지고 소비도 증가하고 있습니다. 미국 정부의 경기부양책과 인프라투자 계획도 경제를 향상시키는 데 도움을 줄 것으로 예상됩니다.

미국 경제의 전망은 여러 요소에 따라 달라질 수 있지만, 현재로서는 경제가 점차 회복되고 있는 모습을 보이고 있습니다. 그러나 여전히 미지수인 요인들도 존재하기 때문에 지켜봐야 할 부분이 많습니다.경제의 변화와 불확실성을 주시하면서 전문가들은 신중하게 경제의 전망을 분석하고 결정하여야 합니다.
------------------------------
미국은 세계 최대의 경제 규모를 가진 나라로, 고도의 산업화와 기술 혁신을 통해 번창하고 있습니다. 미국의 GDP는 세계 최대이며 다양한 산업 부문을 포괄하고 있습니다. 미국의 주요 산업 분야는 기술, 금융, 의료, 자동차, 에너지, 농업 등이 있는데, 특히 기술 분야에서 세계를 선도하고 있습니다.

하지만 미국 경제는 최근 몇 년간 불안정한 요인에 직면해 왔습니다. 보유가 된 환율 문제와 거대한 국가부채, 그리고 중국과의 무역 분쟁 등이 미국 경제를 위협하고 있습니다. 특히, 코로나19 대유행은 미국 경제에도 큰 영향을 미쳤으며, 많은 기업의 폐업과 실업률 증가로 경제의 부정적인 영향을 크게 겪었습니다.

미국은 경제적 자유와 경기 순환의 변동성이 큰 특징을 가지고 있어, 불안정한 경제 상황에 따라 미국의 경제 상황도 변화하고 있습니다. 현재 미국은 백신 접종률이 높아져 경제가 회복되는 모습을 보이고 있지만, 여전히 미지수가 많은 상황이기도 합니다.


# 프롬프트 Template

* PromptTemplate : 프롬프트 template을 정의할 수 있는 Template
* PromptTemplate.from_template : 문자열 template으로부터 PromptTemplate 인스턴스를 생성
* PromptTemplate.format : 템플릿을 채우는 메서드

In [22]:
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

prompt_template = PromptTemplate.from_template(template_text)

filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt


'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [23]:
# 문자열 템플릿 결합
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt.format(name="홍길동", age=30, language="영어")

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

"Hello, my name is Hong Gil-dong and I am 30 years old.\n\nI cannot call my father 'father.'"

# Chat Prompt Template

* ChatPromptTemplate : 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용.
* ChatPromptTemplate.from_messages : 메시지 리스트(혹은 튜플)을 기반으로 프롬프트를 구성함.
* ChatPromptTemplate.format_messages : 사용자의 입력을 프롬프트에 동적으로 삽입하여, 최종적으로 대화형 상황을 반영한 메시지 리스트를 생성

In [24]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
messages

[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]

In [25]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 지름이 약 14만 6천 킬로미터로 태양계의 다른 행성들보다 크기가 가장 큽니다.'

In [34]:
# tuple 대신 MessagePromptTemplate을 사용할 수 있음.

chat_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template("이 시스템은 천문학 질문에 답변할 수 있습니다."), #role을 입력
        HumanMessagePromptTemplate.from_template("{user_input}"), # content (메시지 내용) 을 입력
    ]
)

messages = chat_prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
print(messages)
chain = chat_prompt | llm | StrOutputParser()
chain.invoke(messages)


[SystemMessage(content='이 시스템은 천문학 질문에 답변할 수 있습니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='태양계에서 가장 큰 행성은 무엇인가요?', additional_kwargs={}, response_metadata={})]


'태양계에서 가장 큰 행성은 목성입니다. 목성은 질량과 부피 모두에서 태양계에서 가장 큰 행성이며, 지름은 약 143,000 km로 태양계에서 제일 큽니다.'

In [33]:
chain = chat_prompt | llm | StrOutputParser()

chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

'태양계에서 가장 큰 행성은 목성입니다. 목성은 가장 많은 질량과 부피를 가지고 있어 태양계에서 가장 큰 행성으로 알려져 있습니다.'

# 그 외...

In [35]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 스티브 잡스는 몇 살에 사망했나요?
                  중간 답변: 스티브 잡스는 56세에 사망했습니다.
                  추가 질문: 아인슈타인은 몇 살에 사망했나요?
                  중간 답변: 아인슈타인은 76세에 사망했습니다.
                  최종 답변은: 아인슈타인
                  """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 네이버의 창립자는 누구인가요?
                  중간 답변: 네이버는 이해진에 의해 창립되었습니다.
                  추가 질문: 이해진은 언제 태어났나요?
                  중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
                  최종 답변은: 1967년 6월 22일
                  """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 율곡 이이의 어머니는 누구인가요?
                  중간 답변: 율곡 이이의 어머니는 신사임당입니다.
                  추가 질문: 신사임당은 언제 태어났나요?
                  중간 답변: 신사임당은 1504년에 태어났습니다.
                  추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
                  중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
                  최종 답변은: 연산군
                  """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 올드보이의 감독은 누구인가요?
                  중간 답변: 올드보이의 감독은 박찬욱입니다.
                  추가 질문: 박찬욱은 어느 나라 출신인가요?
                  중간 답변: 박찬욱은 대한민국 출신입니다.
                  추가 질문: 기생충의 감독은 누구인가요?
                  중간 답변: 기생충의 감독은 봉준호입니다.
                  추가 질문: 봉준호는 어느 나라 출신인가요?
                  중간 답변: 봉준호는 대한민국 출신입니다.
                  최종 답변은: 예
                  """,
    },
]

In [36]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"], template="Question: {question}\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question}",
    input_variables=["question"], #프롬프트가 수신할 입력 항목
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"
final_prompt = prompt.format(question=question)
print(final_prompt)

Question: 스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?

                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 스티브 잡스는 몇 살에 사망했나요?
                  중간 답변: 스티브 잡스는 56세에 사망했습니다.
                  추가 질문: 아인슈타인은 몇 살에 사망했나요?
                  중간 답변: 아인슈타인은 76세에 사망했습니다.
                  최종 답변은: 아인슈타인
                  

Question: 네이버의 창립자는 언제 태어났나요?

                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 네이버의 창립자는 누구인가요?
                  중간 답변: 네이버는 이해진에 의해 창립되었습니다.
                  추가 질문: 이해진은 언제 태어났나요?
                  중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
                  최종 답변은: 1967년 6월 22일
                  

Question: 율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?

                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 율곡 이이의 어머니는 누구인가요?
                  중간 답변: 율곡 이이의 어머니는 신사임당입니다.
                  추가 질문: 신사임당은 언제 태어났나요?
                  중간 답변: 신사임당은 1504년에 태어났습니다.
                  추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
                  중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
          

In [38]:
llm = ChatOpenAI(model="gpt-4o-mini")
chain = llm | StrOutputParser()

print(chain.invoke(final_prompt))

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google은 언제 창립되었나요?  
중간 답변: Google은 1998년 9월 4일에 창립되었습니다.  
추가 질문: Bill Gates는 몇 년에 태어났나요?  
중간 답변: Bill Gates는 1955년 10월 28일에 태어났습니다.  
추가 질문: 1998년 9월 4일 기준으로 Bill Gates의 나이는 몇 살인가요?  
중간 답변: 1998년 9월 4일 기준 Bill Gates는 42세입니다.  
최종 답변은: 42세


# 모델 파라미터 주기

In [ ]:
# 모델 생성 단계에서 주기
params = {
    "temperature": 0.7,
    "max_tokens": 100,
}

kwargs = {
    "frequency_penalty": 0.5,
    "presence_penalty": 0.5,
    "stop": ["\n"]

}

model = ChatOpenAI(model="gpt-3.5-turbo-0125", **params, model_kwargs = kwargs)

question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

print(response)

In [ ]:
# 모델 호출 단계에서 주기
params = {
    "temperature": 0.7,
    "max_tokens": 10,
}

response = model.invoke(input=question, **params)

print(response.content)


In [ ]:
# bind 메서드를 통해 모델 인스턴스에 파라미터를 추가로 제공할 수 있음
# 특수한 상황에서만 일부 파라미터를 다르게 적용할 수 있도록 해줌.

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=100)

messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")

before_answer = model.invoke(messages)

print(before_answer)

chain = prompt | model.bind(max_tokens=10)

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})

print(after_answer)

# Parser

*   CSV Parser
  *  CommaSeparatedListOutputParser : 모델이 생성한 텍스트에서 쉼표로 구분된 항목을 추출
  * get_format_instructions : 모델에 전달할 포멧 지시사항을 얻는 메서드
*   Json Parser
  * JsonOutputParser : 모델의 출력을 json으로 해석하며, Pydantic 모델에 맞게 데이터를 구조화하여 제공
  * get_format_instructions : 모델에 전달할 포멧 지시사항을 얻는 메서드



In [42]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

print(format_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [40]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

chain = prompt | llm | output_parser

chain.invoke({"subject": "popular Korean cusine"})


['Bibimbap', 'Kimchi', 'Bulgogi', 'Japchae', 'Tteokbokki']

In [43]:
#자료구조 정의
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine")
    recipe: str = Field(description="recipe to cook the cusine")

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()

print(format_instructions)


The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}
```


In [44]:

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": format_instructions},
)

print(prompt)


input_variables=['query'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of a cusine", "type": "string"}, "recipe": {"title": "Recipe", "description": "recipe to cook the cusine", "type": "string"}}, "required": ["name", "recipe"]}\n```'} template='Answer the user query.\n{format_instructions}\n{query}\n'


In [46]:
chain = prompt | llm | output_parser

chain.invoke({"query": "Let me know how to cook Bibimbap"})

{'name': 'Bibimbap',
 'recipe': 'Bibimbap is a Korean mixed rice dish that is served with an assortment of vegetables, meat, and a spicy sauce. To cook Bibimbap, start by preparing the rice according to package instructions. Then, sauté various vegetables such as spinach, carrots, bean sprouts, and mushrooms separately. Cook some thinly sliced beef in a hot pan with soy sauce and sugar. In a bowl, place the cooked rice at the bottom and arrange the vegetables and beef on top. Add a fried egg and gochujang (Korean chili paste) on the side. Mix everything together before eating. Enjoy your delicious Bibimbap!'}